# Joint Slider Playground (Meshcat)

Use Meshcat sliders to manually set IIWA joint angles and see the robot update live. This notebook builds the same IIWA + WSG model and uses a polling loop to apply slider values to the plant and publish to Meshcat.


In [1]:
import os
import time
import numpy as np
from pathlib import Path

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    DiagramBuilder,
    GeometryInstance,
    MakePhongIllustrationProperties,
    MeshcatVisualizer,
    Parser,
    RenderEngineVtkParams,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    SpatialInertia,
    StartMeshcat,
    UnitInertia,
)

# Meshcat runs in your browser; keep the opened tab to use sliders.



In [2]:
def build_station():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    parser = Parser(plant, scene_graph)

    # Make sure Drake can find manipulation models
    import manipulation
    manipulation_models_path = os.path.join(os.path.dirname(manipulation.__file__), "models")
    parser.package_map().Add("manipulation", manipulation_models_path)

    # Add IIWA
    iiwa_model = parser.AddModelsFromUrl(
        "package://drake_models/iiwa_description/sdf/iiwa14_no_collision.sdf"
    )[0]
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("iiwa_link_0", iiwa_model),
    )

    # Add WSG gripper and weld to iiwa_link_7 (same as main_sim)
    wsg_model = parser.AddModelsFromUrl(
        "package://manipulation/hydro/schunk_wsg_50_with_tip.sdf"
    )[0]
    iiwa_ee_frame = plant.GetFrameByName("iiwa_link_7", iiwa_model)
    wsg_base_frame = plant.GetFrameByName("body", wsg_model)
    rpy = RollPitchYaw(np.deg2rad([90.0, 0.0, 90.0]))
    X_WSG_EE = RigidTransform(rpy.ToRotationMatrix(), p=[0, 0, 0.114])
    plant.WeldFrames(iiwa_ee_frame, wsg_base_frame, X_WSG_EE)

    # Add a small cube on the ground (for reference)
    cube_size = 0.05
    cube_mass = 0.1
    cube_inertia = SpatialInertia(
        mass=cube_mass,
        p_PScm_E=np.zeros(3),
        G_SP_E=UnitInertia.SolidBox(cube_size, cube_size, cube_size),
    )
    cube_body = plant.AddRigidBody("cube", plant.AddModelInstance("Environment"), cube_inertia)
    cube_geom = GeometryInstance(RigidTransform(), Box(cube_size, cube_size, cube_size), "cube_visual")
    cube_geom.set_illustration_properties(
        MakePhongIllustrationProperties(np.array([0.8, 0.2, 0.2, 1.0]))
    )
    plant.RegisterVisualGeometry(cube_body, cube_geom)
    cube_pose = RigidTransform(np.array([0.3, 0.2, cube_size / 2]))
    plant.WeldFrames(plant.world_frame(), cube_body.body_frame(), cube_pose)

    plant.Finalize()

    meshcat = StartMeshcat()
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    # Set a reasonable starting pose (same as current main_sim default)
    q0_iiwa = np.array([1.5, 0.3, 0, 0, 0, -1.5, 1.5])
    q0 = np.zeros(plant.num_positions())
    q0[:7] = q0_iiwa
    plant.SetPositions(plant_context, q0)
    plant.SetVelocities(plant_context, np.zeros(plant.num_velocities()))

    # Publish initial pose
    diagram.ForcedPublish(context)

    return diagram, plant, context, meshcat



In [3]:
# Build the station and start Meshcat

diagram, plant, context, meshcat = build_station()
plant_context = plant.GetMyContextFromRoot(context)

print("Meshcat is running. Open the URL above, then run the next cell for sliders.")


INFO:drake:Meshcat listening for connections at http://localhost:7001


Meshcat is running. Open the URL above, then run the next cell for sliders.


In [4]:
# Create joint sliders (IIWA has 7 joints)
joint_names = [f"q{i+1}" for i in range(7)]
# Use IIWA joint limits from main_sim for slider ranges
q_min = np.array([-2.967, -2.094, -2.967, -2.094, -2.967, -2.094, -3.054])
q_max = np.array([ 2.967,  2.094,  2.967,  2.094,  2.967,  2.094,  3.054])
q_default = plant.GetPositions(plant_context)[:7]

for i, name in enumerate(joint_names):
    meshcat.AddSlider(name=name, min=q_min[i], max=q_max[i], step=0.01, value=float(q_default[i]))

print("Sliders added. Use the next cell to continuously apply slider values.")


Sliders added. Use the next cell to continuously apply slider values.


In [ ]:
# Poll sliders and apply to the robot. Stop with Kernel → Interrupt.

poll_hz = 30.0
poll_dt = 1.0 / poll_hz
print("Polling sliders. Interrupt the kernel to stop.")

while True:
    q = plant.GetPositions(plant_context)
    for i, name in enumerate(joint_names):
        q[i] = meshcat.GetSliderValue(name)
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(context)
    time.sleep(poll_dt)



Polling sliders. Interrupt the kernel to stop.
